In [5]:
# default_exp to_csv

# to csv

> API details.

In [6]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [7]:
# export
import json

In [8]:
# export
import http.server

class MyServer(http.server.BaseHTTPRequestHandler):
  #Handle the web data sent from the strava API
    def do_HEAD(self):
        return self.do_GET()

    def do_GET(self):
        #Get the API code for Strava
#         self.wfile.write('<script>window.close();</script>')
        print(self.path)
        state = {o.split("=")[0]: o.split("=")[1] for o in self.path.split("&")}
        code = state["code"]
        json.dump(state,open('auth_token.json','w'))

In [9]:
# Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client_id, secret = open("../client.secret").read().strip().split(",")

In [10]:
# export
import stravalib
import urllib.parse
import webbrowser
import os
import pandas as pd
import sys
from fastcore.utils import *

In [11]:
httpd = http.server.HTTPServer(("localhost", 9283), MyServer)

In [12]:
httpd.server_close()

In [13]:
# export
class Handel:
    def __init__(self,client_id, secret, port = 5000, url = "http://localhost:xxport/authorized",types = [
            "time",
            "distance",
            "latlng",
            "altitude",
            "velocity_smooth",
            "moving",
            "grade_smooth",
            "temp",
            "watts",
            "cadence",
            "heartrate",
    ]
                ):
        # Create the strava client, and open the web browser for authentication
        self.client_id, self.secret, self.types = client_id, secret, types
        self.client = stravalib.client.Client()
        self.url = url.replace('xxport',str(port))
        self.port = port
        self.authorize_url = self.client.authorization_url(client_id=client_id, redirect_uri=self.url, scope ='activity:read_all')
        
    def get_auth(self,):       
        
        print("Opening: %s" % self.authorize_url)
        webbrowser.open(self.authorize_url)

        try:
            httpd = http.server.HTTPServer(("localhost", self.port), MyServer)
            httpd.handle_request()
            httpd.server_close()

        except KeyboardInterrupt:
            # Allow ^C to interrupt from any thread.
            sys.stdout.write("\033[0m")
            sys.stdout.write("User Interupt\n")
            
    def start(self):
        if not os.path.isfile('auth_token.json'):
            self.get_auth()
        state = json.load(open('auth_token.json','r'))
        if 'access_token' not in state:
            try:
                access_token = self.client.exchange_code_for_token(
                client_id=self.client_id, client_secret=self.secret, code=state['code'])
            except Exception as e:
                print(e,'trying new')
                self.get_auth()
                state = json.load(open('auth_token.json','r'))
                access_token = self.client.exchange_code_for_token(
                client_id=self.client_id, client_secret=self.secret, code=state['code'])
            # access_token save here 
            state['access_token'] = access_token       
            json.dump(state,open('auth_token.json','w'))
            
        # Now store that access token somewhere (for now, it's just a local variable)
        self.client.access_token = state['access_token']["access_token"]
        self.athlete = self.client.get_athlete()
        print(f"Welcome {self.athlete.firstname} {self.athlete.lastname}")

In [27]:
h = Handel(client_id, secret)
h.start()
f"Welcome {h.athlete.firstname} {h.athlete.lastname}"

Welcome Tako Tabak


'Welcome Tako Tabak'

In [15]:
# export
def datetime_fix(start_date, days):
    before = pd.Timestamp(start_date)
    before = pd.Timestamp(before.year,before.month,before.day,23,59)
    before.tz_localize('utc')
    after = before - pd.Timedelta(days=days)
    return before, after

In [16]:
test_eq(datetime_fix(pd.Timestamp('2020'), 7),(pd.Timestamp('2020-01-01 23:59:00'), pd.Timestamp('2019-12-25 23:59:00'))
       )
test_eq(datetime_fix(pd.Timestamp('2020-03-1'), 7),(pd.Timestamp('2020-03-1 23:59:00'), pd.Timestamp('2020-02-23 23:59:00'))
       )

In [24]:
# export
def get_activities(self, start_date = None, days = 7):
    start_date = ifnone(start_date, pd.Timestamp.now())
    before, after = datetime_fix(start_date, days)
    # Returns a list of Strava activity objects, up to the number specified by limit
    activities = self.client.get_activities(before = before, after=after)
#     assert len(list(activities)) == limit
    return {item.name:item for item in activities}

Handel.get_activities = get_activities


In [28]:
h = Handel(client_id, secret)
h.start()
acts = h.get_activities()

In [44]:

print(acts['Avondactiviteit'].start_date,'hr' if acts['Avondactiviteit'].has_heartrate else ' geen Hr' )

2020-02-10 18:02:28+00:00 hr


In [69]:
pd.Timestamp(acts['Schaatstraining'].start_date).date()

datetime.date(2020, 2, 10)

In [58]:
# export
def get_streams(self, activity, types):
    # Returns a Strava 'stream', which is timeseries data from an activity
    streams = self.client.get_activity_streams(activity, types=types, series_type="time")
    return streams
Handel.get_streams = get_streams

def parse_activity(self, act):
    act_id = act.id
    name = act.name
    print(str(act_id), str(act.name), act.start_date)    
    f = str(act_id) + '.csv'
    if not os.path.isfile(f):
        streams = self.get_streams(act_id, self.types)
        df = pd.DataFrame()

        # Write each row to a dataframe
        for item in self.types:
            if item in list(streams.keys()):
                df[item] = pd.Series(streams[item].data, index=None)
            df["act_id"] = act.id
            df["act_startDate"] = pd.to_datetime(act.start_date)
            df["act_name"] = name
        df.to_csv(f, index=False)
    else:
        df = pd.read_csv(f)
    return df, f
Handel.parse_activity = parse_activity

def parse_activitys(self, activities:[], streamlit_progress = None):
    fs = []
    for i, act in enumerate(activities):
        df, f = self.parse_activity(act)
        fs.append(f)
        if streamlit_progress is not None:
            streamlit_progress((i+1)/(len(activities)))
    return fs
Handel.parse_activitys = parse_activitys

In [59]:
h = Handel(client_id, secret)
h.start()
acts = h.get_activities()

AccessUnauthorized: Unauthorized: Authorization Error: [{'resource': 'Athlete', 'field': 'access_token', 'code': 'invalid'}]

In [60]:
p = lambda x: print(x)
fs = h.parse_activitys(acts.values(), streamlit_progress=p)

3088849910 Schaatstraining 2020-02-10 18:02:28+00:00
0.125
3073647207 Middagactiviteit 2020-02-05 12:24:49+00:00
0.25
3085708054 Middagwandeling 2020-02-09 13:29:56+00:00
0.375
3085131773 Herstel werkzaamheden 2020-02-09 11:19:22+00:00
0.5
3081779961 Ochtendrit behoorlijke wind 2020-02-08 08:34:10+00:00
0.625
3077865463 Nachtactiviteit 2020-02-06 20:45:03+00:00
0.75
3078619836 Ochtendactiviteit 2020-02-07 06:04:27+00:00
0.875
3076910670 Middagritje 2020-02-06 13:44:50+00:00
1.0


In [61]:
fs = h.parse_activitys(acts.values())

3088849910 Schaatstraining 2020-02-10 18:02:28+00:00
3073647207 Middagactiviteit 2020-02-05 12:24:49+00:00
3085708054 Middagwandeling 2020-02-09 13:29:56+00:00
3085131773 Herstel werkzaamheden 2020-02-09 11:19:22+00:00
3081779961 Ochtendrit behoorlijke wind 2020-02-08 08:34:10+00:00
3077865463 Nachtactiviteit 2020-02-06 20:45:03+00:00
3078619836 Ochtendactiviteit 2020-02-07 06:04:27+00:00
3076910670 Middagritje 2020-02-06 13:44:50+00:00


In [62]:
from nbdev.export import *
notebook2script()


Converted 01_to_csv.ipynb.
Converted 02_parse.ipynb.
Converted index.ipynb.
